In [1]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import matplotlib.pyplot as plt # 데이터 시각화
import itertools
from datetime import datetime, timedelta # 시간 데이터 처리
from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델
%matplotlib inline


In [2]:
train = pd.read_csv('C:/Users/KIHyuk/Desktop/전력수요예측/train.csv')
test = pd.read_csv('C:/Users/KIHyuk/Desktop/전력수요예측/test.csv')
submission = pd.read_csv('C:/Users/KIHyuk/Desktop/전력수요예측/submission_1002.csv')

test['Time'] = pd.to_datetime(test['Time']) 
test = test.set_index('Time')

In [3]:
place_id=[]; time=[] ; target=[] # 빈 리스트를 생성합니다.
for i in test.columns:
    for j in range(len(test)):
        place_id.append(i) # place_id에 미터 ID를 정리합니다.
        time.append(test.index[j]) # time에 시간대를 정리합니다.
        target.append(test[i].iloc[j]) # target에 전력량을 정리합니다.

new_df=pd.DataFrame({'place_id':place_id,'time':time,'target':target})
new_df=new_df.dropna() # 결측치를 제거합니다.
new_df=new_df.set_index('time') # time을 인덱스로 저장합니다.
new_df.head()

,place_id,target
time,,
2018-04-18 19:00:00,X26,2.527
2018-04-18 20:00:00,X26,2.641
2018-04-18 21:00:00,X26,2.368
2018-04-18 22:00:00,X26,1.899
2018-04-18 23:00:00,X26,1.956


In [4]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
pdq

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [5]:
def get_optimal_params(y):

    param_dict = {}
    for param in pdq:
        try:
            model = ARIMA(y, order=param)
            results_ARIMA = model.fit(disp=-1)
            param_dict[results_ARIMA.aic] = param
        except:
            continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params

In [6]:
agg={}
for key in new_df['place_id'].unique(): # 미터ID 200개의 리스트를 unique()함수를 통해 추출합니다.
    temp = new_df.loc[new_df['place_id']==key] # 미터ID 하나를 할당합니다.
    temp_1h=temp.resample('1h').sum() # 1시간 단위로 정리합니다.
    temp_1day=temp.resample('D').sum() # 1일 단위로 정리합니다.

    # 시간별 예측
    model = ARIMA(temp_1h['target'], order=get_optimal_params(temp_1h['target'])) # AIC를 최소화하는 최적의 파라미터로 모델링합니다.
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(24) # 24시간을 예측합니다.

    a = pd.DataFrame() # a라는 데이터프레임에 예측값을 정리합니다.
    
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.

        
    # 일별 예측
    model = ARIMA(temp_1day['target'], order=get_optimal_params(temp_1day['target'])) # AIC를 최소화하는 최적의 파라미터로 모델링합니다.
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(10) # 10일을 예측합니다.

    for i in range(10):
        a['X2018_7_'+str(i+1)+'_d']=[fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.
    
    
    # 월별 예측
    # 일별로 예측하여 7월 ~ 11월의 일 수에 맞게 나누어 합산합니다.
    fcst = results_ARIMA.forecast(153)
    a['X2018_7_m'] = [np.sum(fcst[0][:31])] # 7월 
    a['X2018_8_m'] = [np.sum(fcst[0][31:62])] # 8월
    a['X2018_9_m'] = [np.sum(fcst[0][62:92])] # 9월
    a['X2018_10_m'] = [np.sum(fcst[0][92:123])] # 10월
    a['X2018_11_m'] = [np.sum(fcst[0][123:153])] # 11월
    
    a['meter_id'] = key 
    agg[key] = a[submission.columns.tolist()]
    print(key)
print('---- Modeling Done ----')

C:\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
C:\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
C:\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which

X26
X303
X241
X435
X402
X352
X305
X350
X326
X299
X160
X125
X211
X349
X407
X330
X132
X231
X225
X16
X286
X354
X353
X76


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X255
X106
X393
X157
X323
X130
X269
X94
X172
X289
X216
X123
X398
X233
X214
X270
X371
X200
X111
X163
X364
X378
X271
X462
X427
X73
X7
X263


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


...

X368
X282
X426


C:\Anaconda3\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:221: RuntimeWarning: divide by zero encountered in true_divide
  Z_mat, R_mat, T_mat)


X47
X192
X283
X329
X223
X266
X20
X443


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X347
X75
X107
X230
---- Modeling Done ----


In [7]:
output1 = pd.concat(agg, ignore_index=False)
output2 = output1.reset_index().drop(['level_0','level_1'], axis=1)
output2['id'] = output2['meter_id'].str.replace('X','').astype(int)
output2 =  output2.sort_values(by='id', ascending=True).drop(['id'], axis=1).reset_index(drop=True)
output2.to_csv('sub_baseline.csv', index=False)

In [8]:
output2

,meter_id,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
0,X5,6.799337,6.608445,6.502227,6.443250,6.410630,6.392714,6.383002,6.377867,6.375286,...,165.487123,167.240644,168.994164,170.747685,172.501205,5673.692192,7358.825438,8725.915274,10674.732795,11934.857877
1,X7,1.308541,1.318726,1.327311,1.334546,1.340644,1.345784,1.350116,1.353767,1.356845,...,40.907214,40.910783,41.008957,41.155804,41.327690,1340.126366,1507.873415,1640.785691,1883.083680,2003.892400
2,X10,2.245585,2.397961,2.522193,2.623480,2.706059,2.773387,2.828279,2.873032,2.909520,...,86.185403,86.498657,86.820665,87.145067,87.470123,2787.029755,3084.692493,3282.837727,3699.838809,3878.140613
3,X16,3.314387,3.423443,3.510585,3.580217,3.635857,3.680316,3.715842,3.744229,3.766913,...,115.646262,116.092716,116.606807,117.153744,117.716633,3774.673963,4311.681047,4701.416845,5404.580433,5759.061412
4,X18,2.495492,2.500715,2.504917,2.508293,2.511005,2.513180,2.514922,2.516315,2.517427,...,69.132699,69.842315,70.551932,71.261548,71.971164,2363.094753,3045.036151,3596.108219,4386.920836,4894.706301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,X468,0.212631,0.211561,0.210810,0.210282,0.209911,0.209650,0.209467,0.209338,0.209248,...,5.020131,5.019469,5.019271,5.019212,5.019194,156.157243,155.594776,150.575589,155.594776,150.575589
196,X470,0.392912,0.390465,0.388141,0.385933,0.383836,0.381844,0.379952,0.378154,0.376447,...,8.250216,8.242835,8.238558,8.236079,8.234642,255.851157,255.212527,246.979865,255.212527,246.979865
197,X472,0.325834,0.331651,0.333931,0.334824,0.335172,0.335307,0.335358,0.335376,0.335381,...,6.119211,6.022374,5.951018,5.898439,5.859696,184.724749,178.287836,172.536133,178.287338,172.536133
198,X473,0.208976,0.195155,0.192387,0.191834,0.191723,0.191702,0.191698,0.191698,0.191698,...,4.194343,4.178976,4.166687,4.156858,4.148998,128.815960,127.646984,123.528231,127.645837,123.528230
